# Embeddingを各テーブルカラムに適用

https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

ユーザーからのクエリ(INPUT)に関連するテーブル情報を抽出するため、カラムごとにEmbeddingベクター情報を事前に付与します。

In [ ]:
# Pythonライブラリインストール
!python --version
!pip install psycopg2-binary
!pip install python-dotenv
!pip install --upgrade openai
!pip install openai[datalib]

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scikit-learn
!pip install sqlalchemy


## 環境変数
supabase接続用URL,APIキーと、openai api接続用のAPIキーを設定します。
自身のopenaiアカウントからapi keyを取得してください。

https://platform.openai.com/account/api-keys

supabaseの情報は管理者にお尋ねください。

下記の例では、.envファイルに変数を書き込んで、JupiterNotebookで読み込む仕様で実装しております。

※.envファイルの作成が困難、.envファイルから値を読み込めない場合、
　os.getenv("◯◯")部分に変数値を直接書き込んでいただいても動作自体には問題ありません。

In [1]:
# 環境変数
import os
from dotenv import load_dotenv
load_dotenv()

# supabase接続用変数
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")

# OPENAI API KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

print('環境変数読み込み完了')

環境変数読み込み完了


# 「table_columns_metadata」テーブルに対してAPIを適用

参考: https://stackoverflow.com/questions/74000154/customize-fine-tune-openai-model-how-to-make-sure-answers-are-from-customized/75192794#75192794

In [5]:
import openai
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
from IPython.display import display

# Use your API key
openai.api_key = openai_api_key

# Connect to the database
connection_config = {
    'user': db_user,
    'password': urllib.parse.quote_plus(db_pass),
    'host': db_host,
    'port': db_port, 
    'database': db_name
}
#sql = 'SELECT * FROM table_column_metadata ORDER BY id ASC;'
sql = 'SELECT * FROM table_column_metadata2 ORDER BY id ASC;'
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))


# Calculate embedding vector for the input using OpenAI Embeddings endpoint
def get_embedding(input: str) -> list[float]:
    result = openai.Embedding.create(
        model = 'text-embedding-ada-002',
        input = input
    )   
    return result['data'][0]['embedding']


# load data from postgre
df = pd.read_sql(sql=sql, con=engine)

df['embedding'] = df['description'].apply(lambda x: get_embedding(x))

display(df)

# INSERT as new rows
df.to_sql('table_column_metadata2', con=engine, if_exists='replace', index=False)


2